In [2]:
import pandas as pd
from glob import glob

In [45]:
trainp = "../data/eleme_round1_train_20200313/%s"
train_actp = glob(trainp%"action/*") #行为数据
train_courp = glob(trainp%"courier/*") #骑手属性数据
train_disp= glob(trainp%"distance/*") #订单距离数据
train_ordp = glob(trainp%"order/*") #订单详细数据

In [ ]:
#距离数据处理

dis = pd.read_csv(train_disp[0])
dis_columns = dis.columns
print(list(dis_columns))
key_index = [list(dis_columns).index(i) for i in ["courier_id","wave_index","tracking_id","source_type","target_tracking_id","target_type"]]
value_index = list(dis_columns).index("grid_distance")
def make_key(x):
    key = "-".join([str(i) for i in x[key_index]])
    value = x[value_index]
    return pd.Series([key,value])
dis_dict = {}
for line in dis.apply(make_key,axis=1).values:
    dis_dict[line[0]] = line[1]
dis_dict

In [42]:
label_list = []
history_list = []
ratio = 0.51
act = pd.read_csv(train_actp[0] #先读取一天的数据
act_group = act.groupby(['courier_id','wave_index'])
for name, group in act_group:
    label_data = group.tail(int(group.shape[0] * ratio))
    history_data = group.drop(label_data.index)
                  #匹配history最后一次行为到label_data里的距离
    histoty_last1 = history_data.iloc[-1,:]
    his_dislast1_key = str(histoty_last1[0])+"-"+str(histoty_last1[1])+"-"+str(histoty_last1[2])+"-"+str(histoty_last1[5])
    label_dis_key = label_data.apply(lambda x:dis_dict.get(his_dislast1_key+"-"+str(x[2])+"-"+str(x[5]),""),axis=1)
    label_data['dis_last1'] = label_dis_key
#     print(label_dis_key)
#     break
    label_data['label'] = 0
    label_data.reset_index(drop=True, inplace=True)
    label_data.loc[0, 'label'] = 1
    label_list.append(label_data)
    history_list.append(history_data)
df_feature_label = pd.concat(label_list, sort=False)
df_his = pd.concat(history_list, sort=False)


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [43]:
df_feature_label.head()

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,dis_last1,label
0,10007871,0,2100074550065333539,121.630997,39.142343,DELIVERY,1580528963,707.0,1
1,10007871,0,2100074550779577850,121.630997,39.142343,PICKUP,1580529129,152.0,0
2,10007871,0,2100074550779577850,121.630997,39.142343,DELIVERY,1580529444,671.0,0
0,10007871,1,2100074555638285402,121.631208,39.142519,PICKUP,1580532225,160.0,1
1,10007871,1,2100074554118800474,121.631208,39.142519,PICKUP,1580532227,160.0,0


In [46]:
history_data.head()

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time
14218,125996858,0,2100074542784775503,121.44622,39.211033,PICKUP,1580519798
14219,125996858,0,2100074544064595093,121.44622,39.211033,PICKUP,1580519902


['courier_id', 'wave_index', 'tracking_id', 'source_type', 'source_lng', 'source_lat', 'target_tracking_id', 'target_type', 'target_lng', 'target_lat', 'grid_distance']


{'100002543-0-2100074536706869066-ASSIGN-2100074542836254538-ASSIGN': 542.0,
 '100002543-0-2100074542836254538-PICKUP-2100074542836254538-ASSIGN': 2042.0,
 '100002543-0-2100074542836254538-DELIVERY-2100074542836254538-ASSIGN': 3343.0,
 '100002543-0-2100074536706869066-PICKUP-2100074542836254538-ASSIGN': 2042.0,
 '100002543-0-2100074536706869066-DELIVERY-2100074542836254538-ASSIGN': 3945.0,
 '100002543-0-2100074542836254538-ASSIGN-2100074542836254538-PICKUP': 1671.0,
 '100002543-0-2100074542836254538-ASSIGN-2100074536706869066-DELIVERY': 3945.0,
 '100002543-0-2100074542836254538-ASSIGN-2100074536706869066-PICKUP': 1671.0,
 '100002543-0-2100074542836254538-ASSIGN-2100074536706869066-ASSIGN': 827.0,
 '100002543-0-2100074542836254538-ASSIGN-2100074542836254538-DELIVERY': 3343.0,
 '100002543-0-2100074536706869066-ASSIGN-2100074542836254538-PICKUP': 1186.0,
 '100002543-0-2100074536706869066-ASSIGN-2100074536706869066-PICKUP': 1186.0,
 '100002543-0-2100074536706869066-ASSIGN-21000745428362545

In [39]:
pd.DataFrame({'a':[1,2,3],'b':[2,3,4]}).iloc[-1,:]

a    3
b    4
Name: 2, dtype: int64

In [ ]:
#关于特征
#0、已经得到label:{0,1}
#1、匹配骑手信息courier数据，得到三个特征 骑手等级、速度、最大接单量
# 骑手等级、速度、最大接单量  label
#  2          4        10       1
#  3          3        5        0
#2/ 匹配订单详细信息数据order,暂时可以得到 天气信息/接单到送单的距离/最后一单的结束时间到这一单承诺送达时间差
#3/ 根据history——data单个骑手的单个波次最后一次送单信息，从distinct数据中匹配label_data中的所有单的距离，已完成